In [1]:
import sys
sys.path.append('..')

import numpy as np
from src.helpers import load_csv_data, standardize, remove_incomplete_columns, predict_labels, create_csv_submission, compute_accuracy
from src.logistic.loss import compute_loss
from src.logistic.not_req_impl import reg_logistic_regression
from src.logistic.gradient import compute_gradient

from src.helpers import remove_correlated_columns, get_jet_indexes, jet_indexes, log_indexes, get_all
from src.polynomials import build_poly_matrix_quadratic
from src.k_fold import build_k_indices

%reload_ext autoreload
%autoreload 2

In [2]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv')
init_col_n = x_raw.shape[1]
init_col_n

30

In [3]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [5]:
x_jet_indexes = get_jet_indexes(x_raw)
# x_jet_indexes = get_all(x_raw)
x = x_raw
# x_mass = np.zeros(x.shape[0])
# x_mass[x[:, 0] == -999] = 1
x[:, 0][x[:, 0] == -999] = np.median(x[:, 0][x[:, 0] != -999])
# x = np.column_stack((x, x_mass))

In [7]:
ws, te_accs, tr_accs, te_losses, tr_losses = [], [], [], [], []
lambda_ = 0
k = 5

for i in x_jet_indexes:
    
    y_i = y[x_jet_indexes[i]]
    tx_i = x[x_jet_indexes[i]]
    tx_del = np.delete(tx_i, jet_indexes[i], axis=1)
    
    for li in log_indexes:
       # print(tx_del[:,li].min())
       tx_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_del[:,li].min()) + n), 0, tx_del[:,li])
    
    tx_std = standardize(tx_del)[0]
    tx_poly = build_poly_matrix_quadratic(tx_std)
    tx = np.c_[np.ones((y_i.shape[0], 1)), tx_poly]
    
    initial_w = np.zeros((tx.shape[1], 1))

    k_indices = build_k_indices(y_i, k, 1)
    
    te_accs_k, tr_accs_k, te_losses_k, tr_losses_k, ws_k = [], [], [], [], []
    
    for k_ in range(k):
        
        test_indices = k_indices[k_]
        train_indices = np.setdiff1d(k_indices.flatten(), test_indices)

        y_train = y_i[train_indices]
        x_train = tx[train_indices]
        y_test = y_i[test_indices]
        x_test = tx[test_indices]

        # Ridge linear
        w, loss_tr_k = reg_logistic_regression(y_train, x_train, initial_w, lambda_, 10000, 0.01, method='sgd', ratio=0.5)

        # Calculate the loss for test data
        loss_te_k = compute_loss(y_test, x_test, w)
        
        acc_tr_k = compute_accuracy(x_train, w, y_train, mode='logistic')
        acc_te_k = compute_accuracy(x_test, w, y_test, mode='logistic')
        
        te_accs_k.append(acc_te_k)
        tr_accs_k.append(acc_tr_k)
        te_losses_k.append(np.math.sqrt(2 * loss_te_k))
        tr_losses_k.append(np.math.sqrt(2 * loss_tr_k))

        ws_k.append(w)
        

    te_accs.append(np.mean(te_accs_k) * x[x_jet_indexes[i]].shape[0])
    tr_accs.append(np.mean(acc_te_k) * x[x_jet_indexes[i]].shape[0])
    te_losses.append(np.mean(te_losses_k) * x[x_jet_indexes[i]].shape[0])
    tr_losses.append(np.mean(tr_losses_k) * x[x_jet_indexes[i]].shape[0])
    ws.append(np.mean(ws_k, axis=0))

    print("lambda={l:.6f}, Training accuracy={tr:.3f}, Testing accuracy={te:.3f}".format(
           l=lambda_, tr=tr_accs[i] / y_i.shape[0], te=te_accs[i] / y_i.shape[0]))

Current iteration=0, loss=[55401.8678478]
||d|| = 2.5787159411771086
Current iteration=1000, loss=[48723.13405209]
||d|| = 5.922103991217265
Current iteration=2000, loss=[49316.23811431]
||d|| = 7.187825262124538
Current iteration=3000, loss=[47112.09800444]
||d|| = 1.2774213848064553
Current iteration=4000, loss=[47590.02996856]
||d|| = 3.5928271124780147
Current iteration=5000, loss=[46988.97556607]
||d|| = 16.16833323616141
Current iteration=6000, loss=[46356.64971734]
||d|| = 0.9629467944887913
Current iteration=7000, loss=[46352.58603949]
||d|| = 5.306026661594372
Current iteration=8000, loss=[46224.20597002]
||d|| = 3.808306576732271
Current iteration=9000, loss=[46568.19341752]
||d|| = 4.211216331407385
loss=[46164.79383636]
Current iteration=0, loss=[55401.8678478]
||d|| = 3.469929013599797
Current iteration=1000, loss=[46925.37148927]
||d|| = 7.224795563582372
Current iteration=2000, loss=[47209.80848357]
||d|| = 4.342158945304195
Current iteration=3000, loss=[46341.56359176]


In [8]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv')
init_col_n = x_raw.shape[1]
init_col_n

30

In [9]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [10]:
x_jet_indexes = get_jet_indexes(x_raw)
# x_jet_indexes = get_all(x_raw)
x = x_raw
x_mass = np.zeros(x.shape[0])
x_mass[x[:, 0] == -999] = 1
x[:, 0][x[:, 0] == -999] = np.median(x[:, 0][x[:, 0] != -999])
x = np.column_stack((x, x_mass))

In [11]:
ws, te_accs, tr_accs, te_losses, tr_losses = [], [], [], [], []
lambda_ = 0
k = 5

for i in x_jet_indexes:
    
    y_i = y[x_jet_indexes[i]]
    tx_i = x[x_jet_indexes[i]]
    tx_del = np.delete(tx_i, jet_indexes[i], axis=1)
    
    for li in log_indexes:
       # print(tx_del[:,li].min())
       tx_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_del[:,li].min()) + n), 0, tx_del[:,li])
    
    tx_std = standardize(tx_del)[0]
    tx_poly = build_poly_matrix_quadratic(tx_std)
    tx = np.c_[np.ones((y_i.shape[0], 1)), tx_poly]
    
    initial_w = np.zeros((tx.shape[1], 1))

    k_indices = build_k_indices(y_i, k, 1)
    
    te_accs_k, tr_accs_k, te_losses_k, tr_losses_k, ws_k = [], [], [], [], []
    
    for k_ in range(k):
        
        test_indices = k_indices[k_]
        train_indices = np.setdiff1d(k_indices.flatten(), test_indices)

        y_train = y_i[train_indices]
        x_train = tx[train_indices]
        y_test = y_i[test_indices]
        x_test = tx[test_indices]

        # Ridge linear
        w, loss_tr_k = reg_logistic_regression(y_train, x_train, initial_w, lambda_, 10000, 0.01, method='sgd', ratio=0.5)

        # Calculate the loss for test data
        loss_te_k = compute_loss(y_test, x_test, w)
        
        acc_tr_k = compute_accuracy(x_train, w, y_train, mode='logistic')
        acc_te_k = compute_accuracy(x_test, w, y_test, mode='logistic')
        
        te_accs_k.append(acc_te_k)
        tr_accs_k.append(acc_tr_k)
        te_losses_k.append(np.math.sqrt(2 * loss_te_k))
        tr_losses_k.append(np.math.sqrt(2 * loss_tr_k))

        ws_k.append(w)
        

    te_accs.append(np.mean(te_accs_k) * x[x_jet_indexes[i]].shape[0])
    tr_accs.append(np.mean(acc_te_k) * x[x_jet_indexes[i]].shape[0])
    te_losses.append(np.mean(te_losses_k) * x[x_jet_indexes[i]].shape[0])
    tr_losses.append(np.mean(tr_losses_k) * x[x_jet_indexes[i]].shape[0])
    ws.append(np.mean(ws_k, axis=0))

    print("lambda={l:.6f}, Training accuracy={tr:.3f}, Testing accuracy={te:.3f}".format(
           l=lambda_, tr=tr_accs[i] / y_i.shape[0], te=te_accs[i] / y_i.shape[0]))

Current iteration=0, loss=[55401.8678478]
||d|| = 3.4241362425983177
Current iteration=1000, loss=[49820.68382129]
||d|| = 12.675298374652264
Current iteration=2000, loss=[47261.71399705]
||d|| = 2.795636048590469
Current iteration=3000, loss=[46613.63864134]
||d|| = 7.766423477049506
Current iteration=4000, loss=[47082.00467998]
||d|| = 11.5013856680073
Current iteration=5000, loss=[46583.53475557]
||d|| = 14.287333971330819
Current iteration=6000, loss=[46299.55575193]
||d|| = 3.871322223252055
Current iteration=7000, loss=[46312.30588604]
||d|| = 1.5363500856181416
Current iteration=8000, loss=[46412.66924231]
||d|| = 1.2021993539628
Current iteration=9000, loss=[46184.71128872]
||d|| = 8.766659316064718
loss=[46976.23589851]
Current iteration=0, loss=[55401.8678478]
||d|| = 10.246944292417446
Current iteration=1000, loss=[48318.82925819]
||d|| = 8.505885622731759
Current iteration=2000, loss=[47809.82206217]
||d|| = 5.960986018032392
Current iteration=3000, loss=[46996.7634807]
||d

In [20]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv')
init_col_n = x_raw.shape[1]
init_col_n

30

In [21]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [22]:
x_jet_indexes = get_jet_indexes(x_raw)
# x_jet_indexes = get_all(x_raw)
x = x_raw
x_mass = np.zeros(x.shape[0])
x_mass[x[:, 0] == -999] = 1
x[:, 0][x[:, 0] == -999] = np.median(x[:, 0][x[:, 0] != -999])
x = np.column_stack((x, x_mass))

In [23]:
ws, te_accs, tr_accs, te_losses, tr_losses = [], [], [], [], []
lambda_ = 0
k = 5

for i in x_jet_indexes:
    
    y_i = y[x_jet_indexes[i]]
    tx_i = x[x_jet_indexes[i]]
    tx_del = np.delete(tx_i, jet_indexes[i], axis=1)
    
    # for li in log_indexes:
    #   # print(tx_del[:,li].min())
    #   tx_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_del[:,li].min()) + n), 0, tx_del[:,li])
    
    tx_std = standardize(tx_del)[0]
    tx_poly = build_poly_matrix_quadratic(tx_std)
    tx = np.c_[np.ones((y_i.shape[0], 1)), tx_poly]
    
    initial_w = np.zeros((tx.shape[1], 1))

    k_indices = build_k_indices(y_i, k, 1)
    
    te_accs_k, tr_accs_k, te_losses_k, tr_losses_k, ws_k = [], [], [], [], []
    
    for k_ in range(k):
        
        test_indices = k_indices[k_]
        train_indices = np.setdiff1d(k_indices.flatten(), test_indices)

        y_train = y_i[train_indices]
        x_train = tx[train_indices]
        y_test = y_i[test_indices]
        x_test = tx[test_indices]

        # Ridge linear
        w, loss_tr_k = reg_logistic_regression(y_train, x_train, initial_w, lambda_, 10000, 0.01, method='sgd', ratio=0.5)

        # Calculate the loss for test data
        loss_te_k = compute_loss(y_test, x_test, w)
        
        acc_tr_k = compute_accuracy(x_train, w, y_train, mode='logistic')
        acc_te_k = compute_accuracy(x_test, w, y_test, mode='logistic')
        
        te_accs_k.append(acc_te_k)
        tr_accs_k.append(acc_tr_k)
        te_losses_k.append(np.math.sqrt(2 * loss_te_k))
        tr_losses_k.append(np.math.sqrt(2 * loss_tr_k))

        ws_k.append(w)
        

    te_accs.append(np.mean(te_accs_k) * x[x_jet_indexes[i]].shape[0])
    tr_accs.append(np.mean(acc_te_k) * x[x_jet_indexes[i]].shape[0])
    te_losses.append(np.mean(te_losses_k) * x[x_jet_indexes[i]].shape[0])
    tr_losses.append(np.mean(tr_losses_k) * x[x_jet_indexes[i]].shape[0])
    ws.append(np.mean(ws_k, axis=0))

    print("lambda={l:.6f}, Training accuracy={tr:.3f}, Testing accuracy={te:.3f}".format(
           l=lambda_, tr=tr_accs[i] / y_i.shape[0], te=te_accs[i] / y_i.shape[0]))

Current iteration=0, loss=[55401.8678478]
||d|| = 6.476739119738675
Current iteration=1000, loss=[38069.98310069]
||d|| = 10.417890193416616
Current iteration=2000, loss=[38339.38567851]
||d|| = 4.652161093775247
Current iteration=3000, loss=[37393.65378116]
||d|| = 2.4425804190750786
Current iteration=4000, loss=[37099.2471176]
||d|| = 1.818833262249398
Current iteration=5000, loss=[36853.63971436]
||d|| = 2.1610816054541933
Current iteration=6000, loss=[36460.39148303]
||d|| = 1.0174059679392067
Current iteration=7000, loss=[36424.98055793]
||d|| = 6.904450684956423
Current iteration=8000, loss=[36196.6889689]
||d|| = 2.1274999535267787
Current iteration=9000, loss=[36449.78074782]
||d|| = 1.6229486924072816
loss=[36227.84596917]
Current iteration=0, loss=[55401.8678478]
||d|| = 4.726917060209904
Current iteration=1000, loss=[38463.39625739]
||d|| = 1.961411138204492
Current iteration=2000, loss=[38102.62244801]
||d|| = 5.856510404057953
Current iteration=3000, loss=[38376.45051467]


In [16]:
# Import data
y, x_raw, ids = load_csv_data('../data/train.csv')
init_col_n = x_raw.shape[1]
init_col_n

30

In [17]:
# Transform y for logistic regression
y[np.where(y == -1)] = 0

In [18]:
x_jet_indexes = get_jet_indexes(x_raw)
# x_jet_indexes = get_all(x_raw)
x = x_raw
# x_mass = np.zeros(x.shape[0])
# x_mass[x[:, 0] == -999] = 1
x[:, 0][x[:, 0] == -999] = np.median(x[:, 0][x[:, 0] != -999])
# x = np.column_stack((x, x_mass))

In [19]:
ws, te_accs, tr_accs, te_losses, tr_losses = [], [], [], [], []
lambda_ = 0
k = 5

for i in x_jet_indexes:
    
    y_i = y[x_jet_indexes[i]]
    tx_i = x[x_jet_indexes[i]]
    tx_del = np.delete(tx_i, jet_indexes[i], axis=1)
    
    # for li in log_indexes:
    #   # print(tx_del[:,li].min())
    #   tx_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_del[:,li].min()) + n), 0, tx_del[:,li])
    
    tx_std = standardize(tx_del)[0]
    tx_poly = build_poly_matrix_quadratic(tx_std)
    tx = np.c_[np.ones((y_i.shape[0], 1)), tx_poly]
    
    initial_w = np.zeros((tx.shape[1], 1))

    k_indices = build_k_indices(y_i, k, 1)
    
    te_accs_k, tr_accs_k, te_losses_k, tr_losses_k, ws_k = [], [], [], [], []
    
    for k_ in range(k):
        
        test_indices = k_indices[k_]
        train_indices = np.setdiff1d(k_indices.flatten(), test_indices)

        y_train = y_i[train_indices]
        x_train = tx[train_indices]
        y_test = y_i[test_indices]
        x_test = tx[test_indices]

        # Ridge linear
        w, loss_tr_k = reg_logistic_regression(y_train, x_train, initial_w, lambda_, 10000, 0.01, method='sgd', ratio=0.5)

        # Calculate the loss for test data
        loss_te_k = compute_loss(y_test, x_test, w)
        
        acc_tr_k = compute_accuracy(x_train, w, y_train, mode='logistic')
        acc_te_k = compute_accuracy(x_test, w, y_test, mode='logistic')
        
        te_accs_k.append(acc_te_k)
        tr_accs_k.append(acc_tr_k)
        te_losses_k.append(np.math.sqrt(2 * loss_te_k))
        tr_losses_k.append(np.math.sqrt(2 * loss_tr_k))

        ws_k.append(w)
        

    te_accs.append(np.mean(te_accs_k) * x[x_jet_indexes[i]].shape[0])
    tr_accs.append(np.mean(acc_te_k) * x[x_jet_indexes[i]].shape[0])
    te_losses.append(np.mean(te_losses_k) * x[x_jet_indexes[i]].shape[0])
    tr_losses.append(np.mean(tr_losses_k) * x[x_jet_indexes[i]].shape[0])
    ws.append(np.mean(ws_k, axis=0))

    print("lambda={l:.6f}, Training accuracy={tr:.3f}, Testing accuracy={te:.3f}".format(
           l=lambda_, tr=tr_accs[i] / y_i.shape[0], te=te_accs[i] / y_i.shape[0]))

Current iteration=0, loss=[55401.8678478]
||d|| = 8.625644532854063
Current iteration=1000, loss=[38411.42511902]
||d|| = 8.294951655184045
Current iteration=2000, loss=[37840.09690223]
||d|| = 3.4985747752798755
Current iteration=3000, loss=[37398.20672033]
||d|| = 1.8017233146020377
Current iteration=4000, loss=[37204.65339111]
||d|| = 10.249003794965903
Current iteration=5000, loss=[37765.08317311]
||d|| = 1.9573549262185745
Current iteration=6000, loss=[36765.68436233]
||d|| = 6.052507492681298
Current iteration=7000, loss=[36769.59367373]
||d|| = 4.6978093253947995
Current iteration=8000, loss=[36557.40860552]
||d|| = 1.2476174996270983
Current iteration=9000, loss=[36562.32876424]
||d|| = 5.455720183403461
loss=[36276.21585838]
Current iteration=0, loss=[55401.8678478]
||d|| = 19.41304749530535
Current iteration=1000, loss=[38749.29699402]
||d|| = 2.0336516351783835
Current iteration=2000, loss=[38340.02406117]
||d|| = 4.891001359336784
Current iteration=3000, loss=[37485.8866449

In [32]:
x = x_raw
x[:, 0][x[:, 0] == -999] = np.median(x[:, 0][x[:, 0] != -999])
y_pred = np.zeros(x.shape[0])

for i, w in enumerate(ws):
    
    tx_i = x[x_jet_indexes[i]]
    tx_del = np.delete(tx_i, jet_indexes[i], axis=1)
    
    for li in log_indexes:
        # print(tx_del[:,li].min())
        tx_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_del[:,li].min()) + n), 0, tx_del[:,li])
    
    tx_std = standardize(tx_del)[0]
    tx_poly = build_poly_matrix_quadratic(tx_std)
    tx = np.c_[np.ones((y_pred[x_jet_indexes[i]].shape[0], 1)), tx_poly]
    
    y_pred[x_jet_indexes[i]] = predict_labels(ws[i].flatten(), tx, mode='logistic')
    
(y == y_pred.reshape(-1, 1)).sum() / y.shape[0]

0.665112

In [ ]:
y_sub, x_sub_raw, ids_sub = load_csv_data('../data/test.csv')
x_sub = x_sub_raw
x_sub[:, 0][x_sub[:, 0] == -999] = np.median(x_sub[:, 0][x_sub[:, 0] != -999])
sub_jet_indexes = get_jet_indexes(x_sub)

In [ ]:
for i, w in enumerate(ws):
    
    tx_i_sub = x_sub[sub_jet_indexes[i]]
    tx_sub_del = np.delete(tx_i_sub, jet_indexes[i], axis=1)
    
    for li in log_indexes:
        # print(tx_del[:,li].min())
        tx_sub_del[:,li] = np.apply_along_axis(lambda n: np.log(1 + abs(tx_sub_del[:,li].min()) + n), 0, tx_sub_del[:,li])
    
    tx_sub_std = standardize(tx_sub_del)[0]
    tx_sub_poly = build_poly_matrix_quadratic(tx_sub_std)
    tx_sub = np.c_[np.ones((y_sub[sub_jet_indexes[i]].shape[0], 1)), tx_sub_poly]
    
    y_sub[x_jet_indexes[i]] = predict_labels(ws[i].flatten, tx_sub, mode='logistic')
    
create_csv_submission(ids_sub, y_sub, '../submissions/10-26.19-57.csv')